In [ ]:
import math, secrets
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit, transpile, ClassicalRegister, QuantumRegister
from qiskit_ibm_runtime import SamplerV2

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Selected backend: {backend.name}")

q = 2**13  
p = 2**10  
T = 2      

def bit_from_counts(bitstring):
    return int(bitstring[0]) 

def quantum_random_int(bits=8):
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    qc = QuantumCircuit(q, c)

    qc.ry(math.pi / 2, q[0])
    if secrets.choice([0, 1]) == 1:
        qc.h(q[0])
    
    qc.measure_all(range(1), range(1))
    qc_transpiled = transpile(qc, backend=backend, optimization_level=3)
    sampler = SamplerV2(mode = backend)
    job = sampler.run([qc_transpiled], shots=5)
    print(f">>> Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()[0]
    samples = result.data.meas.get_counts(0)
    bitstring = result.data.meas.get_bitstrings(0)
    return bit_from_counts(bitstring) 


result = []
for i in range (10):
  result.append(quantum_random_int())


def gen(seed):
    np.random.seed(seed)
    return np.random.randint(0, q, (2, 2))

def beta_mu(shape, seed):
    np.random.seed(seed)
    return np.random.randint(-T, T+1, shape)

def round_mod_q_to_p(x):
    return np.round ((x * p // q)) % p

def keygen():
    seedA = quantum_random_int(8)
    A = gen(seedA)
    seedR = quantum_random_int(8)
    s = beta_mu((2, 1), seedR)
    h = np.full((2, 1), q // 2)
    b = round_mod_q_to_p((A.T @ s + h) % q)
    return (seedA, b), s

def encrypt(pk, m):
    seedA, b = pk
    A = gen(seedA)
    seedE = quantum_random_int(8)
    s = beta_mu((2, 1), seedE)
    h1 = np.full((2, 1), q // 2)
    u = round_mod_q_to_p((A @ s + h1) % q)
    v = ((b.T @ (s % p)) % p).flatten()
    v = np.tile(v, len(m))
    h2 = p // 2
    c = (v + h2 + (m * (p // 2))) % p
    return u, c

def decrypt(sk, c):
    u, v = c
    h2 = p // 2
    v_prime = ((u.T @ (sk % p)) % p).flatten()
    m = ((v - v_prime + h2) % p) // (p // 2)
    return m.astype(int)

pk, sk = keygen()
m = np.array([1,1,0,1,1,0,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,1,1,0,0,0,1,1])
ciphertext = encrypt(pk, m)
decrypted_m = decrypt(sk, ciphertext)
print("Original Message:", m)
print("Encrypted Message:", ciphertext)
print("Decrypted Message:", decrypted_m)


Selected backend: ibm_brisbane
>>> Job ID: cz7jrzk10wx0008c1w10
>>> Job Status: QUEUED
>>> Job ID: cz7jsa5h0kc00089exq0
>>> Job Status: QUEUED
>>> Job ID: cz7jsge39f40008sz7a0
>>> Job Status: QUEUED
>>> Job ID: cz7jsm6p6030008d10f0
>>> Job Status: QUEUED
>>> Job ID: cz7jsvzp6030008d10g0
>>> Job Status: QUEUED
>>> Job ID: cz7jw2g1xvhg008espng
>>> Job Status: QUEUED
>>> Job ID: cz7jw911xvhg008esppg
>>> Job Status: QUEUED
>>> Job ID: cz7jwb91xvhg008espq0
>>> Job Status: QUEUED
>>> Job ID: cz7jwds10wx0008c1wc0
>>> Job Status: QUEUED
>>> Job ID: cz7jwga39f40008sz7pg
>>> Job Status: QUEUED
>>> Job ID: cz7jwjjh0kc00089ey0g
>>> Job Status: QUEUED
>>> Job ID: cz7jwmtkvm9g008gs28g
>>> Job Status: QUEUED
>>> Job ID: cz7jwqakvm9g008gs290
>>> Job Status: QUEUED
Original Message: [1 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1 1]
Encrypted Message: (array([[543],
       [109]]), array([868, 868, 356, 868, 868, 356, 356, 868, 356, 868, 868, 868, 356,
       356, 868, 868, 868, 356, 868, 356, 3